In [1]:
from sklearn import datasets

In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

Загрузка объектов из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм"

In [3]:
newsgroups = datasets.fetch_20newsgroups(subset='all', 
                                         categories=['alt.atheism', 'sci.space'])

In [4]:
# набор данных
X = newsgroups.data

In [5]:
# набор меток классов
Y = newsgroups.target

Можно кодировать тексты, используя TF-IDF. Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency).
Первая равна отношению числа вхождений слова в документ к общей длине документа. 
Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. 
Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

In [6]:
vectorizer = TfidfVectorizer()

In [7]:
# Преобразование обучающей выборки в TF-IDF
# получаем матрицу X (строка - это текст, столбцы - это значение TF-IDF для каждого слова)
X = vectorizer.fit_transform(X)

In [8]:
X.shape

(1786, 28382)

вывод всех слов
print(vectorizer.get_feature_names())

Подбор параметров удобно делать с помощью класса sklearn.model_selection.GridSearchCV

In [9]:
# Подбор параметра C из множества [10^-5, 10^-4, ... 10^4, 10^5]
grid = {'C': np.power(10.0, np.arange(-5, 6))}

# генератор разбиений
cv = KFold(n_splits=5, 
           shuffle=True, 
           random_state=241)

# Классификатор
clf = SVC(kernel='linear', 
          random_state=241)

# Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, 
# вторым — словарь (dict), задающий сетку параметров для перебора
gs = GridSearchCV(clf, 
                  grid, 
                  scoring='accuracy', 
                  cv=cv)

In [10]:
# перебор параметров (в качестве аргументов передаем обработанные тексты и метки ответов)
gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=241, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [11]:
# результаты перебора
gs.cv_results_

{'mean_fit_time': array([3.24639139, 2.96979852, 3.03780189, 2.92542434, 2.45042171,
        1.51960349, 1.51862345, 1.5334291 , 1.523032  , 1.53622088,
        1.53083358]),
 'std_fit_time': array([0.12272881, 0.08505089, 0.02812182, 0.05163215, 0.01282417,
        0.03452071, 0.01835215, 0.03080719, 0.03244785, 0.04223542,
        0.04693114]),
 'mean_score_time': array([0.77521195, 0.71042447, 0.70140157, 0.70639782, 0.60119829,
        0.33221664, 0.33720741, 0.33280802, 0.33137622, 0.33662276,
        0.33679843]),
 'std_score_time': array([0.03050614, 0.00609825, 0.00797167, 0.00833818, 0.02478433,
        0.00999688, 0.01160301, 0.00824949, 0.00878403, 0.01173882,
        0.00721823]),
 'param_C': masked_array(data=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                    1000.0, 10000.0, 100000.0],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False],
        fill_value='?',
             dtype=object)

In [12]:
# лучший результат обучения модели
gs.best_score_

0.9932804406678872

In [13]:
# лучший параметр
best_C = gs.best_params_['C']
best_C

1.0

In [14]:
# обучение классификатора с лучшим параметром С
classifier = SVC(C = best_C, kernel='linear', random_state=241)
classifier.fit(X, Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=241, shrinking=True, tol=0.001,
    verbose=False)

In [15]:
# веса хранятся в поле coef_
type(classifier.coef_)

scipy.sparse.csr.csr_matrix

In [16]:
# превращаем в ndarray (индексы при этом сохраняются)
array = classifier.coef_.toarray()
# берем абсолютные значения коэффициентов
array = abs(array)
array_sorted = sorted(array[0], reverse = True)

In [17]:
# выбираем 10 наибольших коэффициентов
ten_of_the_best = array_sorted[:10]
ten_of_the_best

[2.6631647884797105,
 1.9203794002294927,
 1.2546899512384038,
 1.2491800073760078,
 1.201611181752071,
 1.1801315951388636,
 1.13908083789883,
 1.1306123446649008,
 1.0970936466401477,
 1.0293069271856938]

In [18]:
# по индексам коэффициентов получаем слова, которым классификатор дал наибольший вес
list_of_words = []
for best in ten_of_the_best:
    # в неотсортированном списке находим индексы наибольших коэффициентов
    index_of_word = np.where(array == best)[1][0]
    list_of_words.append(vectorizer.get_feature_names()[index_of_word])
list_of_words = sorted(list_of_words)
list_of_words

['atheism',
 'atheists',
 'bible',
 'god',
 'keith',
 'moon',
 'religion',
 'sci',
 'sky',
 'space']